<h1 align="center">EQE512 MATRIX METHODS IN STRUCTURAL ANALYSIS 
<br>
<br>
Week 06
<br>
<br>
Beam member w/ span load & Column member &  Frame Systems</h1> 

<h3 align="center">Dr. Ahmet Anıl Dindar (adindar@gtu.edu.tr)</h3> 
<h4 align="center">2021 Fall  </h4> 

---
**Today's Topics**

1- Example with beam

2- Column member stiffness matrix

3- Frame System stiffness matrix

---

# 1- Loaded Beam

**For the given system, draw the internal force diagrams. (Material C25)**

<img src="./figures/6-Example-Beam_loaded.png" width = "600">

**Numbering and mapping of the system displacments and forces.**

<img src="./figures/6-Example-Beam_loaded-2.png" width="600">

In [ ]:
DOF = 2

joint_nodes = {1:[0,0] , 2:[0,4], 3:[0,7]}

member_nodes= {1:[1,2] , 2:[2,3]}

member_1 = {"E":30_000_000 ,  "I" : round((.250 * .500**3 )/12 ,10) ,  
                "jointI": joint_nodes[1] , 
                "jointJ": joint_nodes[2]}

member_2 = {"E":30_000_000 ,  "I" : round((.250*.500**3)/12 ,10) ,  
                "jointI": joint_nodes[2] , 
                "jointJ": joint_nodes[3]}

member_props = {}

for member_no , member_property in zip( member_nodes.keys() , [member_1 ,  member_2] ):
    
    member_props[ member_no ] = member_property   
    
member_props

In [ ]:
def beam_load_single_force(f,a,b):
    L = a+b
    s1 = f*( 1 - (3*a**2)/(L**2) + (2*a**3)/(L**3) )
    s2 = f*( a - (2*a**2)/(L) + (a**3)/(L**2) )
    s3 = f*( (3*a**2)/(L**2) - (2*a**3)/(L**3) )
    s4 = f*( -1 * (a**2)/(L) + (a**3)/(L**2) )
    
    return(s1,s2,s3,s4)

In [ ]:
def beam_load_uniform_force(f,L):
    s1 = (f * L) / 2
    s2 = (f * L**2) / 12
    s3 = (f * L) / 2
    s4 = (-1 * f * L**2) / 12    
    
    return(s1,s2,s3,s4)

In [ ]:
# For the member 1
s1_1,s1_2,s1_3,s1_4 = beam_load_single_force( 100 , 2,2)
print( s1_1,s1_2,s1_3,s1_4 )

In [ ]:
# For the member 2
s2_1,s2_2,s2_3,s2_4 = beam_load_uniform_force( 20 , 3)
print( s2_1,s2_2,s2_3,s2_4 )

In [ ]:
# Load Vector

In [ ]:
import numpy as np 

P = np.array([-50 , -50 , -80 , 35, -30 , 15])
print( P )

<img src="./figures/6-Example-Beam_loaded-3.png" width="600">

**Import the previous course notebook**

In [ ]:
import EQE512_functions

In [ ]:
k_members = {}

for member_no , member_property in member_props.items() : 
    print( member_no)
    k_members[member_no] =  EQE512_functions.beam_stiffness_matrix(member_property  )
    
    print( "="*100 , "\n" , k_members[member_no])
    
print( "~"*100)  

In [ ]:
K_system = EQE512_functions.system_stiffness_matrix( k_members , joint_nodes , member_nodes , DOF = 2)
print( K_system)

**Apply Boundary Conditions**

In [ ]:
P_factor = np.array( [ 0 , 0 , 0, 1 , 0 , 1 ] )

In [ ]:
U_factor = np.array( [ 0 , 0 , 0, 1 , 0 , 1 ] )

In [ ]:
K_bc = EQE512_functions.apply_boundary_conditions( K_system , U_factor , P_factor)
K_bc

In [ ]:
P_bc = P*P_factor
P_bc

In [ ]:
U = np.matmul( np.linalg.inv( K_bc ) , P_bc) 
U

**Solution**

In [ ]:
P_solution = np.matmul( K_system , U)
print( P_solution )

In [ ]:
P_support_reactions = P_solution - P
P_support_reactions

---

# Columns

**Freedom**

<img src="figures/9-EQE512-ColumnMemberExplanation.PNG" width="800">

---

**Transformation Matrix**

Remember the relation between local and global terms

$ \Large k^i = (T^i) \hat{k}^i T^i $ 

Remember the transformation matrix;

$ \Large T^i = \begin{bmatrix} \frac{\Delta_1}{L} & \frac{\Delta_2}{L}  & 0 & 0 & 0 & 0 \\ 
                              -\frac{\Delta_2}{L} & \frac{\Delta_1}{L}  & 0  & 0 & 0 & 0 \\ 
                               0 & 0 & 1 & 0 & 0 & 0  \\  
                               0 & 0 & 0 &  \frac{\Delta_1}{L} & \frac{\Delta_2}{L} 0 & 0 &   \\ 
                              0  & 0 & 0 & -\frac{\Delta_2}{L} & \frac{\Delta_1}{L} 0 & 0 &   \\ 
                               0 & 0 & 0 & 0 & 0 & 1  \\  
                               \end{bmatrix}$

---

**Local Stiffness Matrix**

<img src="figures/column_stiffness_in_local_coordinates.png" width="800">

**Global Stiffness Matrix**

After applying the transformation matrix

$ \Large k^i = (T^i)^T \hat{k}^i T^i$

Than the stiffness matrix in global terms

<img src="figures/9-EQE512-ColumnMemberStiffness.PNG" width="1000">

**Transformation Matrix of Frame Element**

In [ ]:
def transformation_matrix( x1,y1,x2,y2) :
    delta_1 = (x1-x2)
    delta_2 = (y1-y2)
    L = ( (delta_1)**2 + (delta_2)**2)**(0.5)
    
    T = np.array( [ ???????? ])
    
    return( T)

**Frame Element Member Stiffness Matrix in Local Coordinates**

In [ ]:
def column_stiffness_matrix():
    ?????????????
    

**Frame Element Member Stiffness Matrix in Global Coordinates**

In [ ]:
K_global =  ????????

---

**NEXT WEEK**
The topics for the next week 

- "Frame Systems- MRF Support displacement"